## Title: Introduction to Approximate Bayesian Computation
### Author: Andrea Giussani
#### Series in "The Long Beard Blog" - Oct 2020

In [1]:
import pandas as pd
import numpy as np

## 1. Dataset Creation

In [2]:
np.random.seed(0)
sales_df = pd.DataFrame(
    {
        "Date": pd.date_range('2020-10-01', periods=5),
        "Lemon": np.random.randint(5,25,size=5),
        "Apple": np.random.randint(5,25,size=5), 
        "Orange": np.random.randint(5,30,size=5),
        "Banana": np.random.randint(5,25,size=5),
        "Mango": np.random.randint(5,20,size=5), 
    }
) 

In [3]:
sales_df.set_index('Date', inplace=True)

In [4]:
sales_df['Total'] = sales_df.sum(axis=1)

In [5]:
sales_df.index.name = None

In [6]:
sales_df

,Lemon,Apple,Orange,Banana,Mango,Total
2020-10-01,17,12,28,6,10,73
2020-10-02,20,14,11,11,14,70
2020-10-03,5,24,29,12,18,88
2020-10-04,8,23,29,19,13,92
2020-10-05,8,9,17,22,14,70


In [7]:
sales_df_copy = sales_df.copy() # just to call it back later

In [8]:
prices_df = pd.DataFrame(
    {
        'item': ['Lemon', 'Apple', 'Orange', 'Banana', 'Mango'],
        'price': [1.2, 0.5, 0.4, 0.7, 1.5]
    }
)


In [9]:
prices_df

,item,price
0,Lemon,1.2
1,Apple,0.5
2,Orange,0.4
3,Banana,0.7
4,Mango,1.5


## 2. The `.eval()` method

In [10]:
for row in prices_df.itertuples():
    col_name = row[1] + ' ' + 'Revenues'
    sales_df[col_name] = pd.eval("sales_df[row[1]]*row[2]")

In [11]:
sales_df

,Lemon,Apple,Orange,Banana,Mango,Total,Lemon Revenues,Apple Revenues,Orange Revenues,Banana Revenues,Mango Revenues
2020-10-01,17,12,28,6,10,73,20.4,6.0,11.2,4.2,15.0
2020-10-02,20,14,11,11,14,70,24.0,7.0,4.4,7.7,21.0
2020-10-03,5,24,29,12,18,88,6.0,12.0,11.6,8.4,27.0
2020-10-04,8,23,29,19,13,92,9.6,11.5,11.6,13.3,19.5
2020-10-05,8,9,17,22,14,70,9.6,4.5,6.8,15.4,21.0


In [12]:
sales_df.loc[:, 'Lemon Revenues':]

,Lemon Revenues,Apple Revenues,Orange Revenues,Banana Revenues,Mango Revenues
2020-10-01,20.4,6.0,11.2,4.2,15.0
2020-10-02,24.0,7.0,4.4,7.7,21.0
2020-10-03,6.0,12.0,11.6,8.4,27.0
2020-10-04,9.6,11.5,11.6,13.3,19.5
2020-10-05,9.6,4.5,6.8,15.4,21.0


In [13]:
sales_df['Total Revenues'] = sales_df.loc[:, 'Lemon Revenues':'Mango Revenues' ].sum(axis=1)

Note that we can get the above results also in this way:
```python
sales_df['Total Revenues'] = pd.eval(
    "sales_df['Lemon Revenues']+ sales_df['Apple Revenues'] + sales_df['Orange Revenues'] + sales_df['Banana Revenues'] + sales_df['Mango Revenues']"
)
```

Note that `.eval()` can be applied to the dataframe dirrectly. In this case, you need to pass the columns' name inside the method. So, for instance, in case you wanted to compute the total sales per day, you just need to sum the columns' name as follows:

In [14]:
sales_df.eval("Total_Eval = Lemon + Apple + Orange + Banana + Mango",inplace=True)

In [15]:
sales_df.loc[:,['Total', 'Total_Eval']]

,Total,Total_Eval
2020-10-01,73,73
2020-10-02,70,70
2020-10-03,88,88
2020-10-04,92,92
2020-10-05,70,70


## 3. The `.query()` method

In [16]:
sales_df = sales_df_copy

In [17]:
sales_df.query("Total >= 80")

,Lemon,Apple,Orange,Banana,Mango,Total
2020-10-03,5,24,29,12,18,88
2020-10-04,8,23,29,19,13,92


In [18]:
condition = 80
sales_df.query("Total >= @condition")

,Lemon,Apple,Orange,Banana,Mango,Total
2020-10-03,5,24,29,12,18,88
2020-10-04,8,23,29,19,13,92


In [19]:
condition = 50
sales_df.query("Total >= @condition & Mango >15")

,Lemon,Apple,Orange,Banana,Mango,Total
2020-10-03,5,24,29,12,18,88


In [20]:
sales_df[(sales_df['Total']>=50)&(sales_df['Mango']>15)]

,Lemon,Apple,Orange,Banana,Mango,Total
2020-10-03,5,24,29,12,18,88


## 4. The `.lookup()` method

In [21]:
np.random.seed(0)
sales_best_df = pd.DataFrame(
    {
        "Date": pd.date_range('2020-10-01', periods=5),
        "Lemon": np.random.randint(5,25,size=5),
        "Apple": np.random.randint(5,25,size=5), 
        "Orange": np.random.randint(5,30,size=5),
        "Banana": np.random.randint(5,25,size=5), 
        "Mango": np.random.randint(5,20,size=5), 
        "Best": ["Orange", "Lemon", "Orange", "Orange", "Banana"]
    }
).set_index('Date')

In [22]:
sales_best_df

,Lemon,Apple,Orange,Banana,Mango,Best
Date,,,,,,
2020-10-01,17,12,28,6,10,Orange
2020-10-02,20,14,11,11,14,Lemon
2020-10-03,5,24,29,12,18,Orange
2020-10-04,8,23,29,19,13,Orange
2020-10-05,8,9,17,22,14,Banana


In [23]:
sales_best_df['Best Quantity'] = sales_best_df.lookup(sales_best_df.index, sales_best_df["Best"])
sales_best_df.loc[:, 'Best':]

,Best,Best Quantity
Date,,
2020-10-01,Orange,28
2020-10-02,Lemon,20
2020-10-03,Orange,29
2020-10-04,Orange,29
2020-10-05,Banana,22


## 5. The `.where()` method

In [24]:
sales_best_df.where(sales_best_df['Best Quantity'] > 25)

,Lemon,Apple,Orange,Banana,Mango,Best,Best Quantity
Date,,,,,,,
2020-10-01,17.0,12.0,28.0,6.0,10.0,Orange,28.0
2020-10-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-10-03,5.0,24.0,29.0,12.0,18.0,Orange,29.0
2020-10-04,8.0,23.0,29.0,19.0,13.0,Orange,29.0
2020-10-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
sales_best_df.where(sales_best_df['Best Quantity'] > 25, 0)['Mango']

Date
2020-10-01    10
2020-10-02     0
2020-10-03    18
2020-10-04    13
2020-10-05     0
Name: Mango, dtype: int64

In [26]:
sales_best_df.mask(sales_best_df['Best Quantity'] > 25, 0)['Mango']

Date
2020-10-01     0
2020-10-02    14
2020-10-03     0
2020-10-04     0
2020-10-05    14
Name: Mango, dtype: int64

## 5. The `.select_dtypes()` method

In [27]:
sales_best_df.dtypes

Lemon             int64
Apple             int64
Orange            int64
Banana            int64
Mango             int64
Best             object
Best Quantity     int64
dtype: object

In [28]:
sales_best_df.loc[:, sales_best_df.dtypes == np.int64]

,Lemon,Apple,Orange,Banana,Mango,Best Quantity
Date,,,,,,
2020-10-01,17,12,28,6,10,28
2020-10-02,20,14,11,11,14,20
2020-10-03,5,24,29,12,18,29
2020-10-04,8,23,29,19,13,29
2020-10-05,8,9,17,22,14,22


Now

In [29]:
sales_best_df.select_dtypes(include=['int64'])

,Lemon,Apple,Orange,Banana,Mango,Best Quantity
Date,,,,,,
2020-10-01,17,12,28,6,10,28
2020-10-02,20,14,11,11,14,20
2020-10-03,5,24,29,12,18,29
2020-10-04,8,23,29,19,13,29
2020-10-05,8,9,17,22,14,22


In [30]:
sales_best_df.select_dtypes(exclude=['object'])

,Lemon,Apple,Orange,Banana,Mango,Best Quantity
Date,,,,,,
2020-10-01,17,12,28,6,10,28
2020-10-02,20,14,11,11,14,20
2020-10-03,5,24,29,12,18,29
2020-10-04,8,23,29,19,13,29
2020-10-05,8,9,17,22,14,22


### END